In [4]:
from transformers import BertModel, BertTokenizer
from datasets import load_dataset
import pandas  as pd
import numpy as np

model_path = "bert-base-uncased"

c:\Users\gkart\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
tokenizer  = BertTokenizer.from_pretrained(model_path)

In [6]:
model = BertModel.from_pretrained(model_path)

In [7]:
dataset = load_dataset('osanseviero/twitter-airline-sentiment')
dataset = dataset["train"].to_pandas()

In [8]:
dataset = dataset[["airline_sentiment","text"]]

In [9]:
dataset['airline_sentiment'] = dataset['airline_sentiment'].map({"positive" : 2, "negative" :1, "neutral" :0})

In [10]:
dataset

,airline_sentiment,text
0,0,@VirginAmerica What @dhepburn said.
1,2,@VirginAmerica plus you've added commercials t...
2,0,@VirginAmerica I didn't today... Must mean I n...
3,1,@VirginAmerica it's really aggressive to blast...
4,1,@VirginAmerica and it's a really big bad thing...
...,...,...
14635,2,@AmericanAir thank you we got on a different f...
14636,1,@AmericanAir leaving over 20 minutes Late Flig...
14637,0,@AmericanAir Please bring American Airlines to...
14638,1,"@AmericanAir you have my money, you change my ..."


In [11]:
model_input = tokenizer("My love is Shree Krishna", return_tensors='pt')

In [12]:
output = model(**model_input)

In [13]:
output

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.4048,  0.1245, -0.0895,  ..., -0.4667,  0.2337,  0.5074],
         [ 0.1067,  0.2586,  0.1721,  ..., -0.2373,  0.3169, -0.0411],
         [ 0.2718,  0.0473,  0.8255,  ..., -0.4655,  0.4836,  0.1226],
         ...,
         [ 0.2828, -0.2119,  0.0163,  ..., -0.1785,  0.0063, -1.8790],
         [-0.2487, -0.9459, -0.2390,  ...,  0.6409,  0.5393, -0.0429],
         [ 0.8047,  0.1197, -0.1789,  ..., -0.1247, -0.7265, -0.3237]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-9.0401e-01, -4.5584e-01, -7.4374e-01,  7.9599e-01,  4.3155e-01,
         -1.1591e-01,  8.8929e-01,  3.0189e-01, -5.0932e-01, -9.9999e-01,
         -1.6417e-01,  8.5574e-01,  9.8759e-01,  3.9256e-01,  9.1653e-01,
         -6.8821e-01, -3.4076e-01, -6.5066e-01,  4.5644e-01, -6.1061e-01,
          6.9212e-01,  9.9988e-01,  3.0251e-01,  2.9379e-01,  5.0830e-01,
          9.6958e-01, -7.5154e-01,  9.1563e-01,  9.6357e-01,  7.248

In [14]:
x_train = dataset['text'].tolist()
y_train = dataset['airline_sentiment'].tolist()

In [15]:
# tokenized_x_train = [tokenizer(i,return_tensors = 'np',padding = True) for i in x_train]

tokenized_x_train = tokenizer(x_train,return_tensors='tf',padding =True)
tokenized_x_train = dict(tokenized_x_train)

In [16]:
tokenized_x_train

{'input_ids': <tf.Tensor: shape=(14640, 67), dtype=int32, numpy=
 array([[  101,  1030,  6261, ...,     0,     0,     0],
        [  101,  1030,  6261, ...,     0,     0,     0],
        [  101,  1030,  6261, ...,     0,     0,     0],
        ...,
        [  101,  1030, 25988, ...,     0,     0,     0],
        [  101,  1030, 25988, ...,     0,     0,     0],
        [  101,  1030, 25988, ...,     0,     0,     0]], dtype=int32)>,
 'token_type_ids': <tf.Tensor: shape=(14640, 67), dtype=int32, numpy=
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>,
 'attention_mask': <tf.Tensor: shape=(14640, 67), dtype=int32, numpy=
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, .

In [17]:
from transformers import TFAutoModelForSequenceClassification
from tensorflow.keras.optimizers import Adam

model = TFAutoModelForSequenceClassification.from_pretrained(model_path,num_labels = 3)
model.compile()

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
model.fit(tokenized_x_train,y_train,batch_size = 4)


 172/3660 [>.............................] - ETA: 31:20 - loss: 1.1573

KeyboardInterrupt: 

In [18]:
y_train = np.array(y_train)

In [74]:
type(y_train)

numpy.ndarray

In [ ]:
from transformers import pipeline
sentiment_analyzer = pipeline("sentiment-analysis")
result= sentiment_analyzer("I love this Airlines")
print(result)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


In [23]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

[]
